In [36]:
import os
from sklearn.preprocessing import LabelEncoder
import cv2
import numpy as np
import tqdm

In [37]:
# list_path = "./food_101/meta/train.txt"
list_path = "./food_101/meta/test.txt"


with open(list_path) as f:
    train_files = f.readlines()
       
train_files = [
    file.strip() for file in train_files
]




In [38]:
def extract_features(image_path, local_feature_type = "sift"):
    # 使用 opencv 提取局部特征的描述子
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if local_feature_type == "sift":
        sift = cv2.xfeatures2d.SIFT_create()
        keypoints, descriptors = sift.detectAndCompute(img, None)
    elif local_feature_type == "orb":
        orb = cv2.ORB_create()
        keypoints = orb.detect(img, None)
        keypoints, descriptors = orb.compute(img, keypoints)
    elif local_feature_type == "surf":
        surf = cv2.xfeatures2d.SURF_create(400)
        keypoints, descriptors = surf.detectAndCompute(img, None)
    return descriptors if descriptors is not None else np.array([])

In [ ]:
data = []
labels = []
local_feature_type = "surf"

image_path = "./food_101/images/"
for index, file_name in enumerate(train_files):
    print(index)
    file_path = os.path.join(image_path, file_name + ".jpg")
    # 提取每张图片的局部特征
    features = extract_features(file_path, local_feature_type=local_feature_type)
    if len(features) > 0:
        data.append(features)
        label = file_name.split("/")[0]
        labels.append(label)
# 如果需要对标签进行编码

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [ ]:
data, labels, label_encoder

In [59]:
from sklearn.cluster import KMeans

def build_visual_vocabulary(data, k=50):
    """
    使用 KMeans 聚类构建视觉词汇, 即 BoW 方法
    """

    # 使用 KMeans 聚类形成视觉词汇
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(data)
    visual_vocabulary = kmeans.cluster_centers_

    return kmeans, visual_vocabulary

In [65]:
def extract_bow_descriptor(data, kmeans, visual_vocabulary):
    # 讲描述子转化为 BoW 向量
    bow_descriptors = []

    for index,descriptor in enumerate(data):
        print(index)
        # 使用 KMeans 模型将每个描述子映射到最近的视觉词汇
        labels = kmeans.predict(descriptor)

        # 统计每个视觉词汇的出现频率
        bow_descriptor = np.bincount(labels, minlength=len(visual_vocabulary))

        # 归一化
        bow_descriptor = bow_descriptor / np.sum(bow_descriptor)

        bow_descriptors.append(bow_descriptor)

    return bow_descriptors

In [62]:
# 训练 kmeans 构造词汇表
stack_train_data = np.vstack(data)
stack_train_data.shape

(1327112, 64)

In [63]:
kmeans, visual_vocabulary = build_visual_vocabulary(stack_train_data, k=50)

In [66]:
bow_des = extract_bow_descriptor(data, kmeans, visual_vocabulary)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [71]:
bow_des

[array([0.02365309, 0.01576873, 0.01839685, 0.01182654, 0.01708279,
        0.03810775, 0.03547963, 0.01445466, 0.01708279, 0.03153745,
        0.02496715, 0.02365309, 0.01971091, 0.02628121, 0.01839685,
        0.01971091, 0.01708279, 0.03679369, 0.0131406 , 0.01971091,
        0.00262812, 0.01839685, 0.00919842, 0.01576873, 0.01576873,
        0.02628121, 0.00919842, 0.02628121, 0.01708279, 0.01445466,
        0.02233903, 0.03153745, 0.02496715, 0.02496715, 0.0131406 ,
        0.03679369, 0.03022339, 0.00131406, 0.02628121, 0.02233903,
        0.01971091, 0.01971091, 0.01839685, 0.02628121, 0.00919842,
        0.01182654, 0.03022339, 0.00394218, 0.00394218, 0.02496715]),
 array([0.01565558, 0.01956947, 0.00391389, 0.02348337, 0.00978474,
        0.00978474, 0.01174168, 0.02544031, 0.11545988, 0.        ,
        0.04305284, 0.01174168, 0.00195695, 0.00391389, 0.00587084,
        0.03131115, 0.01565558, 0.00391389, 0.02935421, 0.01174168,
        0.02544031, 0.02152642, 0.11741683, 0.

In [77]:
arr = np.array(bow_des)
arr.shape

(1500, 50)